# DoIT Schedule Building
### Ray Smith - ECE 524

In [1]:
using Pkg

In [2]:
# Imports
using JuMP
using Clp
using CSV
using DataFrames
using JSON
using OrderedCollections

In [3]:
# Schedule Rules

# General Schedule Layout
hours = 1:32
hours_real = 7:0.5:22.5
days = 1:7

# Shift Types and Training Requirements
shifts = ["HDQA", "Phones", "Chat/Email", "Walk-In"]
training_levels = ["Pick 1", "Walk-In", "Pick 3", "Chat/Email", "HDQA", "SLP"]
requirements = Dict()
requirements[shifts[1]] = [training_levels[5]]
requirements[shifts[2]] = [training_levels[1]]
requirements[shifts[3]] = [training_levels[4]]
requirements[shifts[4]] = [training_levels[2]]

# Allowed Flexibility in (Extra) Agents Scheduled
flexibility = Dict()
flexibility[shifts[1]] = [0]
flexibility[shifts[2]] = [1]
flexibility[shifts[3]] = [1]
flexibility[shifts[4]] = [1]

# Other Rules
adv_phones_req = 1 # min number of pick 3 trained agents scheduled over a phones shift
min_shift_length = 2 # in 1/2 hour blocks

2

In [23]:
# Data Input

# Schedule Shifts
raw_week = CSV.read(joinpath(@__DIR__, "week_req.csv"), DataFrame)
week_shift_types = names(raw_week)
week = Matrix(raw_week)
raw_weekend = CSV.read(joinpath(@__DIR__, "weekend_req.csv"), DataFrame)
weekend_shift_types = names(raw_weekend)
weekend = Matrix(raw_weekend)

# Availability
string_data = join(readlines("availability.json"))
raw_avail = JSON.parse(string_data)
agents = keys(raw_avail)
avail = OrderedDict()
for agent in agents
    raw_agent_avail = raw_avail[agent]
    agent_avail = OrderedDict()
    for i in hours
        agent_avail[i] = raw_agent_avail[i]
    end
    avail[agent] = agent_avail
end

# Training
string_data = join(readlines("agent_training.json"))
raw_trainings = JSON.parse(string_data)
all_agents = keys(raw_trainings)
trainings = OrderedDict()
for agent in all_agents
    raw_agent_trainings = raw_trainings[agent]
    base_job = raw_agent_trainings[:"Base"]
    agent_trainings = raw_agent_trainings[:"Trainings"]
    if startswith(base_job, "SLP")
        append!(agent_trainings, ["SLP"])
    end
    trainings[agent] = agent_trainings
end

In [42]:
# Model Helper Functions

# Check if agent has training
function has_training(agent, training_level)
    return training_level in trainings[agent]
end

# Check if agent meets all training requirements for a shift
function meets_requirements(agent, shift)
    shift_req = requirements[shift]
    for training in shift_req
        if !has_training(agent, training)
            return false
        end
    end
    return true
end

true

In [5]:
# Model Setup

m = Model()

# Solve the max flow problem
set_optimizer(m, optimizer_with_attributes(Clp.Optimizer, "LogLevel"=>0))
optimize!(m)